In [41]:
# coding=utf-8

# @date: 20160902
# @author: DA Popov
# @note: given recent difficulties in implementing the solver KNITRO, it was decided to attempt use of an alternative

## Imports 
import numpy as np
import pandas as pd
from cvxopt import matrix
from cvxopt import solvers
from sklearn.covariance import *

In [42]:
## Functions
# Function prepareProblem: this function constructs the variance-covariance matrix
# @param filePath:
# @param fileName:
# @param shrinkage: whether to apply the Ledoit, Wolf (2004) linear shrinkage estimator
def prepareProblem(filePath, shrinkage=False, subset=False, subsetSize=0):
    # Import data from .csv
    df = pd.read_csv(filePath, sep=';')
    df.index = df.date
    df = df.drop('date', axis=1)

    # Subset, if called via subset == True
    if subset == True:
        df = df.tail(subsetSize)

    # Estimate covariance using Empirical/MLE
    # Expected input is returns, hence set: assume_centered = True
    mleFitted = empirical_covariance(X=df, assume_centered=True)
    sigma = mleFitted

    if shrinkage == True:
        # Estimate covariance using LedoitWolf, first create instance of object
        lw = LedoitWolf(assume_centered=True)
        lwFitted = lw.fit(X=df).covariance_
        sigma = lwFitted

    return sigma

In [43]:
## Main
# read & construct data, matrices etc.
filePath = '/Users/Dim/Desktop/school_folder/masters_thesis/gitCodeRepo/data/noStaleX_returnsData_20160825.csv'
matSigma = prepareProblem(filePath=filePath, shrinkage=False, subset=True, subsetSize=250)
vecVola  = np.sqrt(matSigma.diagonal())
n        = vecVola.shape[0] 

# Check for postive definite covariance matrix, if not exit
if np.all(np.linalg.eigvals(matSigma) > 0) == False:
    print "Sigma matrix not positive definite, solution may not be unique! \nProceed Checking for semi-definiteness\n"
    if np.all(np.linalg.eigvals(matSigma) >= 0) == False:
        print "Sigma matrix not positive semi-definite!"
        raise SystemExit

In [44]:
# Parameterize solver, using standard notation for cvxopt
P = matrix(matSigma,tc='d')
q = matrix(np.zeros(shape=(n, 1)), tc='d')
G = matrix(np.diag(np.repeat(a=-1, repeats=n)), tc='d')
g = matrix(np.zeros(shape=(n, 1)), tc='d')
A = matrix(vecVola.reshape(1,n), tc='d')
a = matrix(np.array([1]), tc='d')

In [45]:
# Invoke solver 
sol = solvers.qp(A=A, b=a, G=G, h=g, P=P, q=q)

     pcost       dcost       gap    pres   dres
 0:  8.9259e-02 -6.5205e+01  7e+01  3e-16  2e+01
 1:  8.9021e-02 -8.8430e-01  1e+00  1e-15  3e-01
 2:  7.7802e-02 -1.4105e-01  2e-01  9e-16  5e-02
 3:  5.9608e-02 -3.0594e-02  9e-02  1e-15  2e-02
 4:  4.4716e-02  1.1967e-02  3e-02  3e-15  2e-03
 5:  3.7833e-02  2.8201e-02  1e-02  3e-15  4e-05
 6:  3.5656e-02  3.3518e-02  2e-03  2e-15  7e-06
 7:  3.4908e-02  3.4471e-02  4e-04  2e-15  1e-06
 8:  3.4713e-02  3.4652e-02  6e-05  2e-15  1e-07
 9:  3.4680e-02  3.4678e-02  2e-06  2e-15  9e-10
10:  3.4679e-02  3.4679e-02  1e-07  2e-15  3e-11
Optimal solution found.


In [46]:
# Solution
xOpt = np.array(sol['x'])
fOpt = np.array(sol['primal objective'])